# Predicting Credit Default Risk to Reduce Financial Losses

## Business Problem
Financial institutions face losses when customers fail to meet their credit obligations.  
The objective of this project is to predict whether a credit card client will default on their payment in the following month, enabling lenders to reduce expected losses, improve risk-based pricing, and intervene early with high-risk customers.

## Stakeholders
- Credit risk and risk analytics teams
- Product managers responsible for pricing and credit limits
- Collections teams prioritising outreach
- Senior management monitoring portfolio risk

## Objective
Build and evaluate classification models that predict the probability of default next month using borrower demographics and credit behaviour data.


# Load libraries and data

In [9]:
!pip install kagglehub

In [10]:
import kagglehub
import pandas as pd
import numpy as np
import os

# Download dataset from Kaggle / UCI
path = kagglehub.dataset_download("uciml/default-of-credit-card-clients-dataset")

# print("Path to dataset files:", path)

os.listdir(path)

df = pd.read_csv(os.path.join(path, "UCI_Credit_Card.csv"))
df.head()


,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,1,20000.0,2,2,1,24,2,2,-1,-1,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,2,2,2,34,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,2,2,1,37,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0
